reference

* https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50
* https://www.kaggle.com/kneroma/m5-forecast-v2-python

* https://www.kaggle.com/iicyan/japanese-m5-forecasting-sell-price-csv

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pylab as plt

In [ ]:
pd.options.display.max_columns = 50

In [ ]:
CAL_DTYPES = {  "event_name_1": "category", "event_type_1": "category", 
                "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
              "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
STE_DTYPES = {  "item_id": "category", "dept_id": "category", "cat_id": "category", "store_id": "category",
              "state_id": "category"  }
PRICE_DTYPES = {  "store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [ ]:
cal = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv", dtype = CAL_DTYPES)
#stv = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_validation.csv")
ste = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv", dtype = STE_DTYPES)
pri = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv", dtype = PRICE_DTYPES)

### calender.csv
* wm_yr_wk: 土曜〜金曜までの一週間 1yywk wkは1~53
* 祝日、催事にはevent_name, typeがつく
* データ数的に365＊7無いので曜日と祝日の掛け合わせを網羅してるわけでは無い
* Snapはどうやら州ごとのフードスタンプ（月100ドル程度の商品券）の許可日っぽい
https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133614

In [ ]:
cal

### sales_train_validation.csv
* item_id store_id と1913日の売上数
* item num = 30490
* 価格決まっていない時（売ってない時）は0padding
* dept_idはFOOD_1 FOOD_2 FOOD_3 HOBBY_1 HOBBY_2 HOUSEHOLD_1 HOUSEHOLD_2

### sales_train_evaluation.csv
* 1941日の売上数　締め切り一ヶ月前に追加 *たぶんこっち使ったほうがいい* 出すのはd_1942~1969

In [ ]:
#stv.head()

In [ ]:
ste

### sell_prices.csv
* 各アイテム各週ごとの価格
* こっちは必ずしもd1から始まるわけでは無い
* 逆に最終日の価格設定は全部あるから多分提出期間中に売ってなかった商品は含まれていないと思われる
* ちょっと検証ががばいけれど多分売り始めたらずっと最後まで売ってる
* 最後に売り始めた商品は11603だから大体200日ぐらいしか売ってない

In [ ]:
pri

In [ ]:
pri[(pri["wm_yr_wk"].diff()!=49) & (pri["wm_yr_wk"].diff()!=48)& (pri["wm_yr_wk"].diff()<0)]["wm_yr_wk"].max()

## 議題
* 商品を売っていない時の0と売ってたけど売れていない0は区別したい（売っていない時はデータに含めないよう）
* 時系列データだが曜日あるいは年週ごとに区切るべきか、区切ると祝日特徴を入れるのが大変そう
* 時系列を考えないとすると曜日flag, 祝日flag, 値段, （商品id）, （店舗id）,
* 出力は売上台数か売上価格か（学習上その議論意味ある？）
* データは店舗でまとめるか商品でまとめるか全部入れるか
* 普通にやるとデータ数の少ない祝日でブレそうなのである程度恣意的に加えたい

## 初期方針
売り場に出した商品の値段含むデータ化をした上で全体、店舗毎、商品（カテゴリ）毎の売り上げ推移可視化

すぐRAMが限界かますので型変換でサイズを落とす

In [ ]:
trans_cols = ["item_id", "dept_id", "cat_id", "store_id", "state_id"]
for col in trans_cols:
    ste[col] = ste[col].cat.codes.astype("int16")
trans_cols = ["item_id", "store_id"]
for col in trans_cols:
    pri[col] = pri[col].cat.codes.astype("int16")
trans_cols = ["event_name_1", "event_type_1"]
for col in trans_cols:
    cal[col] = cal[col].cat.codes.astype("int16") + 1
d_order = {  'Cinco De Mayo': 2, 'Easter': 4, "Father's day": 7, 'OrthodoxEaster': 20  }
cal["event_name_2"] = cal["event_name_2"].map(d_order).fillna(-1).astype("int16") + 1
d_order = {  'Cultural': 0, 'Religious': 2  }
cal["event_type_2"] = cal["event_type_2"].map(d_order).fillna(-1).astype("int16") + 1
cal["date"] = pd.to_datetime(cal["date"])

In [ ]:
cal.drop(['weekday'], axis=1, inplace=True)

In [ ]:
df = pd.melt(ste,id_vars=ste.columns.values[:6],var_name="d",value_name="sells")
df["sells"] = df["sells"].astype("float32")

df = df.merge(cal, on='d', copy = False)
df = df.merge(pri, on=["store_id", "item_id", "wm_yr_wk"],copy = False)
# df = df.sort_values(["wm_yr_wk", "wday"]) # memory over

## 総販売数推移(28日移動平均)

In [ ]:
d_cols = [c for c in ste.columns if 'd_' in c] 

In [ ]:
ste[d_cols].sum().T.plot(kind='line',figsize=(20, 5))
plt.show()

### カテゴリ別

In [ ]:
ste.groupby('cat_id')[d_cols].sum().T.rolling(28).mean().plot(kind='line',figsize=(20, 5))
plt.show()

### 商品種別(dept_id)別

In [ ]:
ste.groupby('dept_id')[d_cols].sum().T.rolling(28).mean().plot(kind='line',figsize=(20, 5))
plt.show()

### 州別

In [ ]:
ste.groupby('state_id')[d_cols].sum().T.rolling(28).mean().plot(kind='line',figsize=(20, 5))
plt.show()

### 店舗別

In [ ]:
ste.groupby('store_id')[d_cols].sum().T.rolling(28).mean().plot(kind='line',figsize=(20, 5))
plt.show()

### 所感
* 全部の売上0になるタイミングが年一で存在する **12/25定休日だった**　対象範囲は5~6月なのでセーフ
* 増加傾向だが対象商品数が増えてるのでまあ当然っちゃ当然
* 店舗毎だとやばそうな増減もちらほらあるけど州では比較的丸まってる

## 対象商品数推移

### カテゴリ別

In [ ]:
df.groupby(['date', 'cat_id']).size().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

### 商品種別(dept_id)別

In [ ]:
df.groupby(['date', 'dept_id']).size().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

### 店舗別

In [ ]:
df.groupby(['date', 'store_id']).size().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

In [ ]:
df[df['store_id']==1].groupby(['date', 'dept_id']).size().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

In [ ]:
df[df['store_id']==7].groupby(['date', 'dept_id']).size().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

In [ ]:
df[df['store_id']==8].groupby(['date', 'dept_id']).size().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

### 所感
* だいたい商品数は3倍増ぐらい
* やっぱ店舗拡大によって一度に大量の商品追加があったパターンがある　これどうしようか
* 追加商品は様々なので店舗ごと切り捨ててしまってもいいかもしれない

## 平均価格推移

### カテゴリ別

In [ ]:
df.groupby(['date', 'cat_id'])["sell_price"].mean().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

### 商品種別(dept_id)別

In [ ]:
df.groupby(['date', 'dept_id'])["sell_price"].mean().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

In [ ]:
df.groupby(['date', 'cat_id'])["sell_price"].sum().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

### 店舗別

In [ ]:
df.groupby(['date', 'store_id'])["sell_price"].mean().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

### 所感
* 2012年夏を境にHOBBY_01が圧倒的に高騰した ならその前はデータに入れないほうが安定しそう
* 単価はFOODが低くあと二つは同じぐらい

## 売上推移

In [ ]:
df1 = pd.concat([df["date"],df["cat_id"],df["dept_id"],df["store_id"],df["state_id"], df["sells"]*df["sell_price"]], axis=1)

### カテゴリ別

In [ ]:
df1.groupby(['date', 'cat_id'])[0].sum().unstack().plot(kind='line',figsize=(20, 5))
plt.show()

In [ ]:
df1.groupby(['date', 'cat_id'])[0].sum().unstack().rolling(28).mean().plot(kind='line',figsize=(20, 5))
plt.show()

### 商品種別(dept_id)別

In [ ]:
df1.groupby(['date', 'dept_id'])[0].sum().unstack().rolling(28).mean().plot(kind='line',figsize=(20, 5))
plt.show()

### 州別

In [ ]:
df1.groupby(['date', 'state_id'])[0].sum().unstack().rolling(28).mean().plot(kind='line',figsize=(20, 5))
plt.show()

### 店舗別

In [ ]:
df1.groupby(['date', 'store_id'])[0].sum().unstack().rolling(28).mean().plot(kind='line',figsize=(20, 5))
plt.show()

### 所感
* これは割と情報量ない（価格平均と販売台数からまあ見える）

## 議論
* データの削除について
 * クリスマスは除いたほうがいい？
 * 平均価格が大きく変化する前は弾きたい
* 特徴量について
 * とりあえず移動平均を与えたい　7日,28日,365日?
 * 商品の追加数は特徴量たり得るか
* 最適化手法について
 * 出力28日分に対して再帰で出すと累積誤差がやばそう